<a href="https://colab.research.google.com/github/msmekka/ncssm-summer25-cyber/blob/main/Secure_AI_Spam_Classifier_Colab_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Secure Your Data, Power Your AI
Welcome to your project notebook! You will build a spam classifier using anonymized data. Work through each section and complete the code where prompted.

# Python Practice

Overview of basic Python concepts

In [ ]:
name = "Ada"
age = 14
height = 5.3

## 🔍 Step 1: Load Your Dataset
We're using the SMS Spam Collection dataset from Kaggle. Upload and load it below.

In [ ]:
# Import libraries
import pandas as pd

filename = 'https://raw.githubusercontent.com/msmekka/ncssm-summer25-cyber/refs/heads/main/spam_clean.csv'

# Load dataset (replace with actual file path or use Kaggle API)
# Import libraries
import pandas as pd

filename = 'https://raw.githubusercontent.com/msmekka/ncssm-summer25-cyber/refs/heads/main/spam_clean.csv'

# Load dataset (replace with actual file path or use Kaggle API)
df = pd.read_csv(filename, sep=',')
#df = pd.read_csv('/content/sample_data/spam.csv', encoding='latin1')
print(df.shape)
df.tail()

(5495, 5)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
5490,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5491,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5492,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5493,ham,The guy did some *****ing but I acted like i'd...,NaN,NaN,NaN
5494,ham,Rofl. Its true to its name,NaN,NaN,NaN


## 🛡️ Step 2: Explore for Sensitive Data
Check for names, numbers, emails, or other personally identifiable information (PII).

In [14]:
# Use regex to identify PII patterns
import re

def find_phone_numbers(text):
  if isinstance(text, str):
    return re.findall(r'\d{11}', text)
    #return re.findall(r'\d{11}|\(?\d{3}[-.\s]?\d{3}[-.\s]?d{4}\)?', text)
  else:
    return []

df['phone_numbers'] = df['v2'].apply(find_phone_numbers)
df['phone_numbers'].head()

#def find_names(text):
#  if isinstance(text, str):
    #Find occurences of the name Chris
    #return re.findall(r'\b[C,c][hris]+\b', text)
    #Find all words that start with a capital letter
#    return re.findall(r'\b[A-Z][a-z]+\b', text)
#  else:
#    return []

df['phone_numbers'] = df['v2'].apply(find_phone_numbers)
df['phone_numbers'].head()
#df['names'] = df['v2'].apply(find_names)
#df['names'].tail()

#How many phone numbers did I find?
#total_phone_numbers = df['phone_numbers'].apply(lambda x: len(x)).sum()
#print(f"Total phone numbers found: {total_phone_numbers}")

#How many names did I find?
#total_names = df['names'].apply(lambda x: len(x)).sum()
#print(f"Total names found: {total_names}")

,phone_numbers
0,[]
1,[]
2,[08452810075]
3,[]
4,[]


## 🔒 Step 3: Anonymize Sensitive Data
Use masking or redaction to protect the data.

In [ ]:
#Anonymize the message
def anonymize_message(msg):
    #msg = re.sub(r'\b\d{11}\b', '[PHONE]', msg)
    msg = re.sub(r'\d{11}', '[PHONE]', msg)
    # Add other anonymization logic here
    return msg

df['v2_anonymized'] = df['v2'].apply(anonymize_message)
#df[['v2', 'message_anonymized']].head()
# Filter for rows that had phone numbers and view the original and anonymized message
df[df['phone_numbers'].apply(lambda x: len(x) > 0)][['v2', 'v2_anonymized']].head()

Total phone numbers found: 429


## 🤖 Step 4: Train a Spam Classifier
Use Scikit-learn to train a simple model on the anonymized messages.

In [ ]:
# Preprocess and vectorize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df['message_anonymized'], df['v1'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       965
        spam       0.98      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.97      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



## 🧾 Step 5: Reflect & Document
Answer the following:
- What types of PII did you find and remove?
- How did you anonymize them?
- Did anonymization affect model accuracy?
- What would the risks be if the data were left unprotected?